In [77]:
# General purpose
import pandas as pd
import os
import glob
import json
import numpy as np
from ast import literal_eval
import string
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
import itertools
from numpy.linalg import norm
from collections import Counter
import codecs
import tqdm
import scipy as sp
from joblib import dump, load
import random
# NLP
from cade.cade import CADE
from gensim.models.word2vec import Word2Vec
# Machine Learning
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.model_selection import cross_val_score
# currently installed theme will be used to
# set plot style if no arguments provided
# Theme
from jupyterthemes import jtplot
jtplot.style()

# Embeddings

Facciamo vedere alcuni esempi di come l'embedding eseguito con CADE, sulle sei diverse testate giornalistiche, dia dei risultati sensati. 

In particolari, si mostrano casi in cui delle stesse parole cambiano la loro rappresentazione (significato) attraverso i 6 diversi modelli; in questi casi, si mostrano dei casi che possono essere in linea con quanto ci si potrebbe aspettare.
È utile mostrare anche dei casi in cui invece si mantiene una parola attraverso i 6 diversi embedding; magari, in tale caso, possiamo usare una parola che appartiene al lessico, e che quindi ci semplifica il lavoro.

Si ricordi che gli embedding non dipendono dal *lessico* scelto, e di conseguenza sono agnostici ad essi. L'unica cosa che li determina sono i corpora, che, a meno di errori nella pulizia, e.g. avremmo potuto eseguire della *lemmitization* o cose simili.

Questi confronti presuppongono che Wikipedia sia la fonte più "da dizionario" e oggettive tra quelle proposte. Di conseguenza, si usa la sua rappresentazione per mettere a confronto quelle degli altri.

In [3]:
slices = {filename.split('/')[-1].replace(".model", ""): 
          Word2Vec.load(filename)
          for filename in glob.glob('./models/*.model')}

In [23]:
for sl in slices:
    print(sl)
    print(slices["Wikipedia"].wv.most_similar([slices[sl].wv["politics"]])[0])

Wikipedia
('politics', 1.0)
ABC News
('politics', 0.558868408203125)
Breitbart
('philosophy', 0.5695109367370605)
CNN
('politics', 0.6155990958213806)
The Federalist
('politics', 0.5537101030349731)
New York Times
('politics', 0.6331716179847717)
News Max
('politics', 0.5335795879364014)


Come si può notare, la rappresentazione della parola *politics* in quasi tutti i giornali coincide con la rappresentazione all'interno di wikipedia; l'unico che è differente è Breitbart, nel qual il concetto corrisponde più al *philosophy*.

In [34]:
for sl in slices:
    print(sl)
    print(slices["Wikipedia"].
          wv.most_similar([slices[sl].wv["progressive"]])[0:2])

Wikipedia
[('progressive', 0.9999999403953552), ('radical', 0.7494416832923889)]
ABC News
[('progressive', 0.6467471718788147), ('liberal', 0.6081336736679077)]
Breitbart
[('progressive', 0.6694939732551575), ('radical', 0.6292732954025269)]
CNN
[('progressive', 0.688573956489563), ('liberal', 0.6536697149276733)]
The Federalist
[('progressive', 0.5990332961082458), ('radical', 0.5633199214935303)]
New York Times
[('progressive', 0.595812201499939), ('liberal', 0.5684530735015869)]
News Max
[('progressive', 0.6231521368026733), ('liberal', 0.571533203125)]


Questo dà risultati molto interessanti. Come si può notare, la rappresentazione della parola *progressive* è condivisa attraverso tutto gli embedding; quello che però cambia è la seconda parola più simile: se si prende la rappresentazione di un giornale "di sinistra", *progressive* tende a essere più vicino al concetto di *liberal* (in Wikipedia); mentre per giornali di destra estrema è più simile a *radical*.

In [52]:
for sl in slices:
    print(sl)
    print(slices["Wikipedia"].
          wv.most_similar([slices[sl].wv["illegal"]])[0:2])

Wikipedia
[('illegal', 1.0), ('unlawful', 0.72054123878479)]
ABC News
[('illegal', 0.7067723870277405), ('unlawful', 0.5750272274017334)]
Breitbart
[('illegal', 0.5317680239677429), ('undocumented', 0.47897130250930786)]
CNN
[('illegal', 0.6380700469017029), ('unlawful', 0.5811516046524048)]
The Federalist
[('illegal', 0.6033549308776855), ('undocumented', 0.46863484382629395)]
New York Times
[('illegal', 0.7472089529037476), ('unlawful', 0.6054619550704956)]
News Max
[('illegal', 0.6044532656669617), ('unlawful', 0.5079938173294067)]


Anche in questo caso, si nota un pattern simile ai precedenti: i giornali di destra tendono ad avere il significato di *illegal* più vicino ad *undocumented* (in Wikipedia), rispetto agli altri che sono su *unlawful*.

In [74]:
for sl in slices:
    print(sl)
    print(slices["Wikipedia"].
          wv.most_similar([slices[sl].wv["god"]])[0:2])

Wikipedia
[('god', 1.0), ('gods', 0.8261622190475464)]
ABC News
[('god', 0.5821242332458496), ('lover', 0.5465329885482788)]
Breitbart
[('god', 0.6881555914878845), ('jesus', 0.6249061822891235)]
CNN
[('oh', 0.6258580684661865), ('god', 0.5774052143096924)]
The Federalist
[('god', 0.5874472260475159), ('thy', 0.515599250793457)]
New York Times
[('god', 0.6862984895706177), ('heaven', 0.5891343355178833)]
News Max
[('god', 0.6187192797660828), ('heaven', 0.5652803182601929)]


Questa piccolo esempio è interessante: la parola ha quasi la stessa rappresentazione per tutti, ma sono leggermente spostasti: la rappresentazione, per esempio, si CNN è più vicina a 

In [77]:
for sl in slices:
    print(sl)
    print(slices["Wikipedia"].
          wv.most_similar([slices[sl].wv["good"]])[0:3])

Wikipedia
[('good', 1.0), ('bad', 0.7421819567680359), ('fun', 0.6268191337585449)]
ABC News
[('good', 0.6347428560256958), ('bad', 0.46216851472854614), ('keen', 0.4531018137931824)]
Breitbart
[('good', 0.6908650398254395), ('fun', 0.5131772756576538), ('remarkable', 0.5018529891967773)]
CNN
[('good', 0.6638422608375549), ('bad', 0.5046796798706055), ('fun', 0.5020245313644409)]
The Federalist
[('good', 0.6423277854919434), ('bad', 0.44417208433151245), ('keen', 0.41980552673339844)]
New York Times
[('good', 0.7027113437652588), ('keen', 0.5303717851638794), ('bad', 0.49873054027557373)]
News Max
[('good', 0.6140873432159424), ('bad', 0.5044747591018677), ('fun', 0.4753373861312866)]


Questa parola rimane costante, anche nella seconda più vicina.

# Propagations

## Without Propagation

Qua si mostrano semplicemente degli esempi di parole che sono state selezionate dopo la procedura di LRP.

In [8]:
with open("./lexicon/lexicon_refined.csv") as file:
    lexicon_refined = pd.read_csv(file)
lexicon_refined = lexicon_refined.rename(columns={'Unnamed: 0': "Word"})

In [51]:
lexicon_refined[lexicon_refined["Word"] == "bad"]

,Word,Valence


In [52]:
lexicon_refined[lexicon_refined["Word"] == "good"]

,Word,Valence
18,good,-1.0


In [15]:
print("###########")
print("5 subjective words in the lexicon refined.")
print(lexicon_refined[lexicon_refined["Valence"] == 1]
      .sample(5)["Word"]
      .to_list())

###########
5 subjective words in the lexicon refined.
['little', 'will', 'court', 'long', 'like']


In [16]:
print("###########")
print("5 objective words in the lexicon refined.")
print(lexicon_refined[lexicon_refined["Valence"] == -1]
      .sample(5)["Word"]
      .to_list())

###########
5 objective words in the lexicon refined.
['least', 'point', 'back', 'so', 'support']


## Hamilton

In [54]:
# We have to remove some nan, because Hamilton sucks
propagations = {filename.split("_")[-1].replace(".csv", ""):
                (pd.read_csv(filename, index_col=1).dropna()
                .drop(columns="Unnamed: 0").to_dict()["Labels"])
                for filename in glob.glob('./propagations/*.csv')}

In [79]:
# Finding some words
for name, prop in propagations.items():
    print("###", name, "###")
    print(random.sample([name for name, val in prop.items() if val > 0.9], 
                       5))

### Wikipedia ###
['cartoon', 'crowther', 'javanicum', 'uproar', 'tate']
### Breitbart ###
['railed', 'olly', 'though', 'commenting', 'nhl']
### New York Times ###
['uptodate', 'todaypresident', 'rr', 'wolcott', 'hoosier']
### News Max ###
['begged', 'revelation', 'georgetowns', 'case', 'identity']
### CNN ###
['cafeteria', 'extensions', 'describes', 'fingernails', 'permeated']
### The Federalist ###
['felon', 'orchestrated', 'twoweek', 'ideals', 'compost']
### ABC News ###
['overriding', '230c', 'markedly', 'defy', 'midterm']


In [83]:
test_words = ["tyrants", "decisively", 
              "redeem", "unacceptable", "snow"]
CORRECTLY_PROPAGATED = "snow"
WRONGLY_PROPAGATED = "snow"

In [57]:
for name, prop in propagations.items():
    print("###", name, "###")
    print(CORRECTLY_PROPAGATED, "{:0.2f}".format(prop[CORRECTLY_PROPAGATED]))

### Wikipedia ###
bad 0.26
### Breitbart ###
bad 0.66
### New York Times ###
bad 0.12
### News Max ###
bad 0.06
### CNN ###
bad 0.93
### The Federalist ###
bad 0.13
### ABC News ###
bad 0.31


## Nicoli Logistic

In [31]:
slices = {filename.split('/')[-1].replace(".model", ""): 
          Word2Vec.load(filename)
          for filename in glob.glob('./models/*.model')}

In [63]:
inducer = load('./ML_models/Nicoli_logistic.joblib')
models_propagation_nicoli = {name: {word: 
                       (inducer.predict_proba([model.wv[word]])[0][1])
  for word in model.wv.vocab} for name, model in slices.items()}

## Together

In [90]:
CORRECTLY_PROPAGATED = "good"

In [91]:
print("^^^^ NICOLI ^^^^")
for name, dic in models_propagation_nicoli.items():
    print("###", name, '###')
    print(CORRECTLY_PROPAGATED, "{:0.4f}".format(dic[CORRECTLY_PROPAGATED]))

^^^^ NICOLI ^^^^
### Wikipedia ###
good 0.1151
### ABC News ###
good 0.0019
### Breitbart ###
good 0.7160
### CNN ###
good 0.7916
### The Federalist ###
good 0.0000
### New York Times ###
good 0.0014
### News Max ###
good 0.3850


In [92]:
print("^^^^ HAMILTON ^^^^")
for name, prop in propagations.items():
    print("###", name, "###")
    print(CORRECTLY_PROPAGATED, "{:0.2f}".format(prop[CORRECTLY_PROPAGATED]))

^^^^ HAMILTON ^^^^
### Wikipedia ###
good 0.00
### Breitbart ###
good 0.00
### New York Times ###
good 0.01
### News Max ###
good 0.00
### CNN ###
good 0.05
### The Federalist ###
good 0.00
### ABC News ###
good 0.01
